# Морфологический разбор старославянского языка при помощи DistilBERT

Задача: сделать pos-теггер для старославянского.

Данные: старославянский treebank от Universal Dependencies.

Наши данные находятся в формате CONLLU:

In [1]:
with open('/content/drive/MyDrive/ML_training_data/chu_all.conllu', 'r') as f:
  chu_lines = f.readlines()

In [2]:
for line in chu_lines[:10]:
  print(line)

# source = Codex Marianus, Matthew 9

# text = ꙇ вьлѣзъ въ корабь и҃съ прѣѣде ꙇ приде въ свои градъ

# sent_id = 38541

1	ꙇ	и	CCONJ	C-	_	6	cc	_	ref=MATT_9.1

2	вьлѣзъ	вълѣсти	VERB	V-	Case=Nom|Gender=Masc|Number=Sing|Strength=Strong|Tense=Past|VerbForm=Part|Voice=Act	6	advcl	_	ref=MATT_9.1

3	въ	въ	ADP	R-	_	4	case	_	ref=MATT_9.1

4	корабь	корабль	NOUN	Nb	Case=Acc|Gender=Masc|Number=Sing	2	obl	_	ref=MATT_9.1

5	и҃съ	исоусъ	PROPN	Ne	Case=Nom|Gender=Masc|Number=Sing	6	nsubj	_	ref=MATT_9.1

6	прѣѣде	прѣꙗхати	VERB	V-	Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	ref=MATT_9.1

7	ꙇ	и	CCONJ	C-	_	6	cc	_	ref=MATT_9.1



Поэтому для начала надо их предобработать и достать из них только слова и частеречные теги.

In [3]:
import re

In [4]:
def read_corp(file_path):

  with open(file_path, 'r') as f:
    raw_text = f.read()

  raw_docs = re.split(r'\n\t?\n', raw_text)
  token_docs = []
  tag_docs = []
  for doc in raw_docs:
      tokens = []
      tags = []
      for line in doc.split('\n'):
        if line:
          if line.split()[0] != '#':
            info = line.split('\t')
            tokens.append(info[1])
            tags.append(info[3])
      token_docs.append(tokens)
      tag_docs.append(tags)

  return token_docs, tag_docs

texts, tags = read_corp('/content/drive/MyDrive/ML_training_data/chu_all.conllu')

In [5]:
len(texts)

6339

In [6]:
texts[0]

['ꙇ',
 'вьлѣзъ',
 'въ',
 'корабь',
 'и҃съ',
 'прѣѣде',
 'ꙇ',
 'приде',
 'въ',
 'свои',
 'градъ']

In [7]:
tags[0]

['CCONJ',
 'VERB',
 'ADP',
 'NOUN',
 'PROPN',
 'VERB',
 'CCONJ',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN']

In [8]:
train_texts = texts[:5071]
train_tags = tags[:5071]
val_texts = texts[5071:]
val_tags = tags[5071:]

Сделаем словарь всех уникальных тегов и каждому из них присвоим номер, чтобы можно было легко кодировать и декодировать теги:

In [9]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [10]:
tag2id

{'CCONJ': 0,
 'ADJ': 1,
 'VERB': 2,
 'AUX': 3,
 'X': 4,
 'INTJ': 5,
 'NUM': 6,
 'PROPN': 7,
 'SCONJ': 8,
 'ADV': 9,
 'DET': 10,
 'PRON': 11,
 'NOUN': 12,
 'ADP': 13}

In [11]:
! pip install datasets transformers[torch] seqeval accelerate -U

Мы будем работать с DistilBERT - более легковесной версией BERT. Также мы постараемся побольше использовать функционал библиотеки transformers.

In [12]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 4

In [13]:
# AutoTokenizer сам определяет, какой токенайзер взять
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Кодируем теги частей речи перед подачей в модель:

In [14]:
encoded_tags_train = [[tag2id[tag] for tag in doc] for doc in train_tags]

In [15]:
encoded_tags_val = [[tag2id[tag] for tag in doc] for doc in val_tags]

Нам нужно, чтобы количество лейблов соответствовало количеству токенов в закодированном предложении (т.е. с уже добавленным паддингом).

In [16]:
label_all_tokens = True

def tokenize_and_align_labels(texts, tags):

    tokenized_inputs = tokenizer(texts,
                                 truncation=True, # обрезка слишком длинных последовательностей
                                 padding=True,
                                 is_split_into_words=True # предупреждаем, что вход поступит в виде списков токенов
                                 )

    labels = []
    for i, label in enumerate(tags):

        # достаем 1 текст
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        # идем по всем словам
        for word_idx in word_ids:

            # Некоторые специальные токены имеют id None. Мы даем им лейбл -100, чтобы модель их игнорировала
            if word_idx is None:
                label_ids.append(-100)

            # Логично, что если слово разделилось на subword-токены, их лейблы в пределах слова должны быть одинаковыми.
            # Если мы перешли на новое слово, добавляем его лейбл в список
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            # Всем следующим частям одного и того же слова мы даем или тот же лейбл, или -100, если label_all_tokens=False
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    return tokenized_inputs, labels

In [17]:
tokenized_train, train_labels = tokenize_and_align_labels(train_texts, encoded_tags_train)

In [18]:
print(train_labels[10])

[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [19]:
tokenized_val, val_labels = tokenize_and_align_labels(val_texts, encoded_tags_val)

In [20]:
import torch

Как и pytorch, transformers может оперировать сущностями класса Dataset. Напишем такой класс, который не будет ничего делать, кроме хранения токенизированных текстов и их лейблов и подачи их в модель.

In [21]:
class CHUDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      # этот метод вызывается моделью, когда она учится
      # он определяет, в каком виде данные подаются в модель
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
train_dataset = CHUDataset(tokenized_train, train_labels)
val_dataset = CHUDataset(tokenized_val, val_labels)

In [23]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Сначала объявим аргументы, которые будет использовать модель при тренировке:

In [24]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"//content/sample_data/{model_name}-finetuned-1-pos-chu",
    overwrite_output_dir=True, # записываем каждый раз в один и тот же файл
    evaluation_strategy = "epoch", # оцениваем каждую эпоху
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, # размер тренировочного батча на каждый процессор
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01, # регуляризация функции потерь
    push_to_hub=False, # не публиковать на Huggingface
)

Коллаторы данных - это технические классы, формирующие батчи для подачи в модель:

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
from datasets import load_metric

In [27]:
metric = load_metric("seqeval")

<ipython-input-27-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Мы будем использовать метрику для оценки NER-моделей, которая возвращает precision, recall и f1-score для каждой сущности (в нашем случае - для каждого тега) и средние значения.

In [28]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Удаляем индексы специальных токенов
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Тренировка модели в transformers удобно осуществляется при помощи класса Trainer.

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.736100,0.584016,0.659152,0.609794,0.633513,0.813834
2,0.460300,0.476431,0.696994,0.647556,0.671366,0.855249


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CCONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:17

KeyboardInterrupt: 

Оценка модели на валидационных данных:

In [31]:
predictions, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions, axis=2)

# Уберем игнорируемые токены и декодируем предсказанные токены
ids_predictions = [
    [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=ids_predictions, references=true_labels)
results

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.736100,0.584016,0.659152,0.609794,0.633513,0.813834
2,0.460300,0.476431,0.696994,0.647556,0.671366,0.855249


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CCONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:17

{'CONJ': {'precision': 0.64258962011771,
  'recall': 0.7875409836065573,
  'f1': 0.7077195050088391,
  'number': 1525},
 'DJ': {'precision': 0.6553459119496855,
  'recall': 0.5173783515392254,
  'f1': 0.5782463928967815,
  'number': 1007},
 'DP': {'precision': 0.967020023557126,
  'recall': 0.9535423925667829,
  'f1': 0.960233918128655,
  'number': 861},
 'DV': {'precision': 0.87115165336374,
  'recall': 0.7353224254090471,
  'f1': 0.7974947807933194,
  'number': 1039},
 'ERB': {'precision': 0.5894090560245587,
  'recall': 0.6626402070750647,
  'f1': 0.6238830219333875,
  'number': 2318},
 'ET': {'precision': 0.4444444444444444,
  'recall': 0.38620689655172413,
  'f1': 0.4132841328413284,
  'number': 145},
 'NTJ': {'precision': 0.8363636363636363,
  'recall': 0.7666666666666667,
  'f1': 0.8,
  'number': 60},
 'OUN': {'precision': 0.6747603833865815,
  'recall': 0.6200822078684675,
  'f1': 0.6462668298653611,
  'number': 1703},
 'RON': {'precision': 0.7801998183469573,
  'recall': 0.610

Попробуем загрузить и использовать модель с чекпойнта:

In [32]:
from transformers import TokenClassificationPipeline

In [33]:
# здесь можно поставить путь к файлу чекпойнта или адрес модели на huggingface
#checkpoint = 'annadmitrieva/old-church-slavonic-pos'
checkpoint = '/content/sample_data/distilbert-base-uncased-finetuned-1-pos-chu/checkpoint-3000'

In [34]:
tokenizer_chu = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
model_chu = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [36]:
chubert = TokenClassificationPipeline(model=model_chu, tokenizer=tokenizer_chu, task="pos")

In [40]:
def postprocess_output(output):

  string = ''
  tags = []
  last_token_end = 0

  for token in output:
    # сначала обработаем pos-тег
    if '_' in token['entity']:
      pos = id2tag[int(token['entity'].split('_')[1])]
    else:
      pos = token['entity']

    if token['word'][0] != '#':
      # если токен - или начало слова, или полное слово
      if last_token_end != token['start']:
        string += ' '

      string += token['word']
      tags.append(pos)

    else:
      # если токен - середина слова или конец
      string += token['word'].replace('#', '')

    last_token_end = token['end']

  return list(zip(string.split(), tags))

In [38]:
chubert('нъ да оувѣсте ѣко власть иматъ с҃нъ ч҃лвчскꙑ на земи отъпоущати грѣхꙑ')

[{'entity': 'LABEL_0',
  'score': 0.9890698,
  'index': 1,
  'word': 'н',
  'start': 0,
  'end': 1},
 {'entity': 'LABEL_0',
  'score': 0.9897579,
  'index': 2,
  'word': '##ъ',
  'start': 1,
  'end': 2},
 {'entity': 'LABEL_8',
  'score': 0.8944431,
  'index': 3,
  'word': 'д',
  'start': 3,
  'end': 4},
 {'entity': 'LABEL_8',
  'score': 0.9214833,
  'index': 4,
  'word': '##а',
  'start': 4,
  'end': 5},
 {'entity': 'LABEL_2',
  'score': 0.6475895,
  'index': 5,
  'word': 'оувѣсте',
  'start': 6,
  'end': 13},
 {'entity': 'LABEL_2',
  'score': 0.23805733,
  'index': 6,
  'word': 'ѣко',
  'start': 14,
  'end': 17},
 {'entity': 'LABEL_12',
  'score': 0.9962909,
  'index': 7,
  'word': 'в',
  'start': 18,
  'end': 19},
 {'entity': 'LABEL_12',
  'score': 0.9967313,
  'index': 8,
  'word': '##л',
  'start': 19,
  'end': 20},
 {'entity': 'LABEL_12',
  'score': 0.99668103,
  'index': 9,
  'word': '##а',
  'start': 20,
  'end': 21},
 {'entity': 'LABEL_12',
  'score': 0.99639845,
  'index': 10,

In [41]:
postprocess_output(chubert('нъ да оувѣсте ѣко власть иматъ с҃нъ ч҃лвчскꙑ на земи отъпоущати грѣхꙑ'))

[('нъ', 'CCONJ'),
 ('да', 'SCONJ'),
 ('оувѣсте', 'VERB'),
 ('ѣко', 'VERB'),
 ('власть', 'NOUN'),
 ('иматъ', 'VERB'),
 ('снъ', 'NOUN'),
 ('ч҃лвчскꙑ', 'ADJ'),
 ('на', 'ADP'),
 ('земи', 'NOUN'),
 ('отъпоущати', 'VERB'),
 ('грѣхꙑ', 'PRON')]